In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

import numpy as np
from matplotlib import pyplot as plt
import math
import statistics as stat

import sqlalchemy
from sqlalchemy import create_engine
import psycopg2
from config import db_password

import datetime
from datetime import date
from datetime import time

import re

## Read Kaggle data from matches.csv

Import the table `matches.csv` from Kaggle data on major league soccer (MLS).

In [2]:
# Read .csv from FinalProject/Resources
file_dir = "../Resources/Clean_csv/matches_cleaned.csv"    

# Create a pandas DataFrame 
game_df = pd.read_csv(file_dir, low_memory=False, skipinitialspace = True)

# Show all columns
pd.set_option("display.max_columns", None)

game_df

,Unnamed: 0,Unnamed: 0.1,id,home,away,year,attendance,venue,game_status,shootout,home_score,away_score,home_possessionPct,away_possessionPct,home_foulsCommitted,away_foulsCommitted,home_yellowCards,away_yellowCards,home_redCards,away_redCards,home_offsides,away_offsides,home_wonCorners,away_wonCorners,home_saves,away_saves,Outcome
0,0,1797,336076,Colorado Rapids,Columbus Crew SC,2012,14746.0,"Dick's Sporting Goods Park, Denver",FT,False,2,0,0.53,0.47,11,14,1,3,0,0,2,4,8,4,3,3,Win
1,1,1798,336077,Vancouver Whitecaps,Montreal Impact,2012,21000.0,"BC Place, Vancouver",FT,False,2,0,0.50,0.50,13,26,1,2,0,0,2,2,4,4,7,3,Win
2,2,1799,336078,DC United,Sporting Kansas City,2012,16314.0,"RFK Stadium, Washington, D.C.",FT,False,0,1,0.43,0.57,12,16,1,1,0,0,3,0,3,5,7,1,Loss
3,3,1800,336080,LA Galaxy,Real Salt Lake,2012,27000.0,"Dignity Health Sports Park, Los Angeles",FT,False,1,3,0.49,0.51,14,10,0,2,0,0,3,4,7,2,2,3,Loss
4,4,1801,336079,San Jose Earthquakes,New England Revolution,2012,10525.0,Buck Shaw Stadium,FT,False,1,0,0.52,0.48,17,14,2,1,0,0,2,3,3,3,1,0,Win
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3115,3429,5226,597845,Vancouver Whitecaps,Real Salt Lake,2021,12009.0,"Rio Tinto Stadium, Sandy, UT",FT,False,0,4,0.55,0.45,7,9,0,2,0,0,0,0,5,4,4,2,Loss
3116,3430,5227,597846,LA Galaxy,FC Dallas,2021,16453.0,"Dignity Health Sports Park, Carson",FT,False,3,1,0.53,0.47,7,15,1,3,0,0,0,3,2,2,3,2,Win
3117,3431,5228,597848,Nashville SC,Atlanta United FC,2021,22913.0,"Nissan Stadium, Nashville, Nashville, TN",FT,False,2,2,0.50,0.50,16,17,3,4,0,1,6,1,6,3,0,6,Tie
3118,3432,5229,597847,New York Red Bulls,Philadelphia Union,2021,10013.0,"Red Bull Arena, Harrison",FT,False,1,1,0.47,0.53,14,9,2,2,0,1,1,4,5,5,5,2,Tie


In [3]:
game_df = game_df.drop(["game_status", "shootout"], axis='columns')
game_df.head(2)

,Unnamed: 0,Unnamed: 0.1,id,home,away,year,attendance,venue,home_score,away_score,home_possessionPct,away_possessionPct,home_foulsCommitted,away_foulsCommitted,home_yellowCards,away_yellowCards,home_redCards,away_redCards,home_offsides,away_offsides,home_wonCorners,away_wonCorners,home_saves,away_saves,Outcome
0,0,1797,336076,Colorado Rapids,Columbus Crew SC,2012,14746.0,"Dick's Sporting Goods Park, Denver",2,0,0.53,0.47,11,14,1,3,0,0,2,4,8,4,3,3,Win
1,1,1798,336077,Vancouver Whitecaps,Montreal Impact,2012,21000.0,"BC Place, Vancouver",2,0,0.50,0.50,13,26,1,2,0,0,2,2,4,4,7,3,Win


In [4]:
game_df = game_df.drop(["home_possessionPct", "home_score", "away_score", "home_foulsCommitted", "home_yellowCards", "home_redCards", "home_offsides", "home_wonCorners", "home_saves"], axis='columns')
game_df.head(2)

,Unnamed: 0,Unnamed: 0.1,id,home,away,year,attendance,venue,away_possessionPct,away_foulsCommitted,away_yellowCards,away_redCards,away_offsides,away_wonCorners,away_saves,Outcome
0,0,1797,336076,Colorado Rapids,Columbus Crew SC,2012,14746.0,"Dick's Sporting Goods Park, Denver",0.47,14,3,0,4,4,3,Win
1,1,1798,336077,Vancouver Whitecaps,Montreal Impact,2012,21000.0,"BC Place, Vancouver",0.50,26,2,0,2,4,3,Win


In [5]:
game_df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'id', 'home', 'away', 'year',
       'attendance', 'venue', 'away_possessionPct', 'away_foulsCommitted',
       'away_yellowCards', 'away_redCards', 'away_offsides', 'away_wonCorners',
       'away_saves', 'Outcome'],
      dtype='object')

In [5]:
game_df.isnull().sum()

Unnamed: 0             0
Unnamed: 0.1           0
id                     0
home                   0
away                   0
year                   0
attendance             0
venue                  0
away_possessionPct     0
away_foulsCommitted    0
away_yellowCards       0
away_redCards          0
away_offsides          0
away_wonCorners        0
away_saves             0
Outcome                0
dtype: int64

In [6]:
game_df.dtypes

Unnamed: 0               int64
Unnamed: 0.1             int64
id                       int64
home                    object
away                    object
year                     int64
attendance             float64
venue                   object
away_possessionPct     float64
away_foulsCommitted      int64
away_yellowCards         int64
away_redCards            int64
away_offsides            int64
away_wonCorners          int64
away_saves               int64
Outcome                 object
dtype: object

In [7]:
# No. of venues before removing duplicate venue names
game_df.nunique()

Unnamed: 0             3120
Unnamed: 0.1           3120
id                     3120
home                     30
away                     31
year                     10
attendance             2452
venue                    63
away_possessionPct       57
away_foulsCommitted      29
away_yellowCards          8
away_redCards             4
away_offsides            12
away_wonCorners          18
away_saves               14
Outcome                   3
dtype: int64

It looks like there are 4 columns with **categorical** values having consistent data type as *object*.

We should **generate a list of categorical variable** names using Python's "df.dtypes" property. In that case, we can use our variable list to perform the **one-hot encoding** *once*, rather than for each individual variable.

In [8]:
# Generate our categorical variable list
game_cat = game_df.dtypes[game_df.dtypes == "object"].index.tolist()
game_cat

['home', 'away', 'venue', 'Outcome']

In [9]:
# Check the number of unique values in each column
game_df[game_cat].nunique()

home       30
away       31
venue      63
Outcome     3
dtype: int64

In [10]:
game_df['home'].unique()

array(['Colorado Rapids', 'Vancouver Whitecaps', 'DC United', 'LA Galaxy',
       'San Jose Earthquakes', 'FC Dallas', 'Chivas USA',
       'Portland Timbers', 'Montreal Impact', 'Sporting Kansas City',
       'Real Salt Lake', 'Seattle Sounders FC', 'Philadelphia Union',
       'Toronto FC', 'Columbus Crew SC', 'New England Revolution',
       'Chicago Fire FC', 'New York Red Bulls', 'Houston Dynamo',
       'Orlando City SC', 'New York City FC', 'Atlanta United FC',
       'Minnesota United FC', 'LAFC', 'FC Cincinnati',
       'Houston Dynamo FC', 'Nashville SC', 'Inter Miami CF',
       'Columbus Crew', 'Austin FC'], dtype=object)

Before going to encode these columns using Scikit-learn's OneHotEncoder module, we need to make sure that if the categorical variables require **bucketing**. Since we want to predict outcome for each individual `home` team, we need to see the relation of all individual `home` team, `venue` and `game_status`, we choose not to to any binning/bucketing, and we're ready to use **OneHotEncoder**.

In [11]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(game_df[game_cat]))
encode_df.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [12]:
# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(game_cat)
encode_df.head()

,home_Atlanta United FC,home_Austin FC,home_Chicago Fire FC,home_Chivas USA,home_Colorado Rapids,home_Columbus Crew,home_Columbus Crew SC,home_DC United,home_FC Cincinnati,home_FC Dallas,home_Houston Dynamo,home_Houston Dynamo FC,home_Inter Miami CF,home_LA Galaxy,home_LAFC,home_Minnesota United FC,home_Montreal Impact,home_Nashville SC,home_New England Revolution,home_New York City FC,home_New York Red Bulls,home_Orlando City SC,home_Philadelphia Union,home_Portland Timbers,home_Real Salt Lake,home_San Jose Earthquakes,home_Seattle Sounders FC,home_Sporting Kansas City,home_Toronto FC,home_Vancouver Whitecaps,away_Atlanta United FC,away_Austin FC,away_CF MontrÃ©al,away_Chicago Fire FC,away_Chivas USA,away_Colorado Rapids,away_Columbus Crew,away_Columbus Crew SC,away_DC United,away_FC Cincinnati,away_FC Dallas,away_Houston Dynamo,away_Houston Dynamo FC,away_Inter Miami CF,away_LA Galaxy,away_LAFC,away_Minnesota United FC,away_Montreal Impact,away_Nashville SC,away_New England Revolution,away_New York City FC,away_New York Red Bulls,away_Orlando City SC,away_Philadelphia Union,away_Portland Timbers,away_Real Salt Lake,away_San Jose Earthquakes,away_Seattle Sounders FC,away_Sporting Kansas City,away_Toronto FC,away_Vancouver Whitecaps,venue_Allianz Field,"venue_Allianz Field, Minnesota",venue_Audi Field,"venue_Audi Field, Washington DC",venue_BBVA Stadium,"venue_BBVA Stadium, Houston, TX","venue_BC Place, Vancouver",venue_BMO Field,venue_Banc of California Stadium,"venue_Banc of California Stadium, Los Angeles",venue_Bobby Dodd Stadium,venue_Buck Shaw Stadium,"venue_Camping World Stadium, Orlando","venue_CenturyLink Field, Seattle","venue_Children's Mercy Park, Kansas City","venue_Children's Mercy Park, Kansas City, KS",venue_Citi Field Stadium,"venue_DRV PNK Stadium, Fort Lauderdale, FL","venue_Dick's Sporting Goods Park, Commerce City, CO","venue_Dick's Sporting Goods Park, Denver","venue_Dignity Health Sports Park, Carson","venue_Dignity Health Sports Park, Los Angeles",venue_Earthquakes Stadium,venue_Exploria Stadium,"venue_Exploria Stadium, Orlando, Florida",venue_FedExField,venue_Gillette Stadium,"venue_Historic Crew Stadium, Columbus, OH",venue_Inter Miami CF Stadium,venue_Levi's Stadium,"venue_Lower.com Field, Columbus, OH","venue_Lumen Field, Seattle","venue_MAPFRE Stadium, Columbus","venue_Maryland SoccerPlex, Germantown",venue_Mercedes-Benz Stadium,"venue_Mercedes-Benz Stadium, Atlanta, Georgia",venue_Navy-Marine Corps Memorial Stadium,venue_Nippert Stadium,"venue_Nissan Stadium, Nashville","venue_Nissan Stadium, Nashville, Nashville, TN","venue_PayPal Park, San Jose",venue_Pratt & Whitney Stadium at Rentschler Field,"venue_Providence Park, Portland","venue_Q2 Stadium, Austin","venue_RFK Stadium, Washington, D.C.",venue_Red Bull Arena,"venue_Red Bull Arena, Harrison","venue_Rio Tinto Stadium, Sandy, UT","venue_Rio Tinto Stadium, Utah",venue_Rogers Centere,venue_SeatGeek Stadium,"venue_Soldier Field, Chicago",venue_Stade Olympique,venue_Stade Saputo,venue_Stanford Stadium,"venue_Subaru Park, Chester","venue_Subaru Park, Philadelphia",venue_TCF Bank Stadium,"venue_TQL Stadium, Cincinnati, OH",venue_Toyota Field,venue_Toyota Stadium,"venue_Toyota Stadium, Frisco, TX",venue_Yankee Stadium,Outcome_Loss,Outcome_Tie,Outcome_Win
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.

Now that our categorical variables have been encoded, they are ready to replace our unencoded categorical variables in our dataset using *panda's* **merge** and **drop** methods.

In [13]:
# Merge OneHotEncoded features and drop the originals from the updated df
game_df = game_df.merge(encode_df, left_index=True, right_index=True).drop(game_df[game_cat],1)
game_df.head()

,Unnamed: 0,Unnamed: 0.1,id,year,attendance,away_possessionPct,away_foulsCommitted,away_yellowCards,away_redCards,away_offsides,away_wonCorners,away_saves,home_Atlanta United FC,home_Austin FC,home_Chicago Fire FC,home_Chivas USA,home_Colorado Rapids,home_Columbus Crew,home_Columbus Crew SC,home_DC United,home_FC Cincinnati,home_FC Dallas,home_Houston Dynamo,home_Houston Dynamo FC,home_Inter Miami CF,home_LA Galaxy,home_LAFC,home_Minnesota United FC,home_Montreal Impact,home_Nashville SC,home_New England Revolution,home_New York City FC,home_New York Red Bulls,home_Orlando City SC,home_Philadelphia Union,home_Portland Timbers,home_Real Salt Lake,home_San Jose Earthquakes,home_Seattle Sounders FC,home_Sporting Kansas City,home_Toronto FC,home_Vancouver Whitecaps,away_Atlanta United FC,away_Austin FC,away_CF MontrÃ©al,away_Chicago Fire FC,away_Chivas USA,away_Colorado Rapids,away_Columbus Crew,away_Columbus Crew SC,away_DC United,away_FC Cincinnati,away_FC Dallas,away_Houston Dynamo,away_Houston Dynamo FC,away_Inter Miami CF,away_LA Galaxy,away_LAFC,away_Minnesota United FC,away_Montreal Impact,away_Nashville SC,away_New England Revolution,away_New York City FC,away_New York Red Bulls,away_Orlando City SC,away_Philadelphia Union,away_Portland Timbers,away_Real Salt Lake,away_San Jose Earthquakes,away_Seattle Sounders FC,away_Sporting Kansas City,away_Toronto FC,away_Vancouver Whitecaps,venue_Allianz Field,"venue_Allianz Field, Minnesota",venue_Audi Field,"venue_Audi Field, Washington DC",venue_BBVA Stadium,"venue_BBVA Stadium, Houston, TX","venue_BC Place, Vancouver",venue_BMO Field,venue_Banc of California Stadium,"venue_Banc of California Stadium, Los Angeles",venue_Bobby Dodd Stadium,venue_Buck Shaw Stadium,"venue_Camping World Stadium, Orlando","venue_CenturyLink Field, Seattle","venue_Children's Mercy Park, Kansas City","venue_Children's Mercy Park, Kansas City, KS",venue_Citi Field Stadium,"venue_DRV PNK Stadium, Fort Lauderdale, FL","venue_Dick's Sporting Goods Park, Commerce City, CO","venue_Dick's Sporting Goods Park, Denver","venue_Dignity Health Sports Park, Carson","venue_Dignity Health Sports Park, Los Angeles",venue_Earthquakes Stadium,venue_Exploria Stadium,"venue_Exploria Stadium, Orlando, Florida",venue_FedExField,venue_Gillette Stadium,"venue_Historic Crew Stadium, Columbus, OH",venue_Inter Miami CF Stadium,venue_Levi's Stadium,"venue_Lower.com Field, Columbus, OH","venue_Lumen Field, Seattle","venue_MAPFRE Stadium, Columbus","venue_Maryland SoccerPlex, Germantown",venue_Mercedes-Benz Stadium,"venue_Mercedes-Benz Stadium, Atlanta, Georgia",venue_Navy-Marine Corps Memorial Stadium,venue_Nippert Stadium,"venue_Nissan Stadium, Nashville","venue_Nissan Stadium, Nashville, Nashville, TN","venue_PayPal Park, San Jose",venue_Pratt & Whitney Stadium at Rentschler Field,"venue_Providence Park, Portland","venue_Q2 Stadium, Austin","venue_RFK Stadium, Washington, D.C.",venue_Red Bull Arena,"venue_Red Bull Arena, Harrison","venue_Rio Tinto Stadium, Sandy, UT","venue_Rio Tinto Stadium, Utah",venue_Rogers Centere,venue_SeatGeek Stadium,"venue_Soldier Field, Chicago",venue_Stade Olympique,venue_Stade Saputo,venue_Stanford Stadium,"venue_Subaru Park, Chester","venue_Subaru Park, Philadelphia",venue_TCF Bank Stadium,"venue_TQL Stadium, Cincinnati, OH",venue_Toyota Field,venue_Toyota Stadium,"venue_Toyota Stadium, Frisco, TX",venue_Yankee Stadium,Outcome_Loss,Outcome_Tie,Outcome_Win
0,0,1797,336076,2012,14746.0,0.47,14,3,0,4,4,3,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1

We need to **split** our **training** and **testing** data *before* fitting our **StandardScaler** instance. This <u> prevents testing data from influencing the standardization </u> function.

To build our training and testing datasets, we need to separate two values:

input values (which are our *independent variables* commonly referred to as **model features or "X"**) and **target output** ( *dependent variable* commonly referred to as **target or "y"** in TensorFlow documentation).

We want to build a model that will predict whether or not a team is winning; therefore, we must separate the `Outcome_win` column from the rest of the input data. Also, the `Outcome_loss`, `Outcome_tie`columns have the same input for all the data, so we can drop this column.

In [14]:
# Split our preprocessed data into our features and target arrays
y = game_df["Outcome_Win"].values
X = game_df.drop(["Outcome_Win","Outcome_Loss", "Outcome_Tie" ], 1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

Now that our training and testing data have been allocated, we're ready to build our **StandardScalerobject** and standardize the numerical features.

In [15]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Our data is now **preprocessed** via one-hot encoding and standardization. 

### Define Neural Network model

For our **input layer**, we must add the **number of input features equal to the number of variables in our feature** DataFrame.

In our **hidden layers**, we'll add **three hidden layers** with only a few neurons in each layer. To create the *second hidden layer*, we'll add another **Keras Dense class** while defining our model. All of our hidden layers will use the **relu activation** function to identify nonlinear characteristics from the input values.

In the **output layer**, we'll use the `sigmoid` activation function that will help us predict the probability that a team is winning or not.

In [16]:
len(X_train[0])

136

In [17]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  272
hidden_nodes_layer2 = 136
hidden_nodes_layer3 = 44

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 272)               37264     
_________________________________________________________________
dense_1 (Dense)              (None, 136)               37128     
_________________________________________________________________
dense_2 (Dense)              (None, 44)                6028      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 45        
Total params: 80,465
Trainable params: 80,465
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
#nn.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
74/74 [==============================] - 1s 5ms/step - loss: 0.6998 - accuracy: 0.5538
Epoch 2/100
74/74 [==============================] - 0s 5ms/step - loss: 0.6222 - accuracy: 0.6530
Epoch 3/100
74/74 [==============================] - 0s 4ms/step - loss: 0.5831 - accuracy: 0.6940
Epoch 4/100
74/74 [==============================] - 0s 4ms/step - loss: 0.5386 - accuracy: 0.7316
Epoch 5/100
74/74 [==============================] - 0s 5ms/step - loss: 0.5010 - accuracy: 0.7654
Epoch 6/100
74/74 [==============================] - 0s 5ms/step - loss: 0.4617 - accuracy: 0.7795
Epoch 7/100
74/74 [==============================] - 0s 5ms/step - loss: 0.4346 - accuracy: 0.7902
Epoch 8/100
74/74 [==============================] - 0s 5ms/step - loss: 0.4000 - accuracy: 0.8141
Epoch 9/100
74/74 [==============================] - 0s 5ms/step - loss: 0.3676 - accuracy: 0.8410
Epoch 10/100
74/74 [==============================] - 0s 7ms/step - loss: 0.3287 - accuracy: 0.8645
Epoch 11/

74/74 [==============================] - 1s 12ms/step - loss: 7.6418e-04 - accuracy: 1.0000
Epoch 79/100
74/74 [==============================] - 1s 12ms/step - loss: 9.4839e-04 - accuracy: 1.0000 0s - loss: 4
Epoch 80/100
74/74 [==============================] - 1s 11ms/step - loss: 5.0638e-04 - accuracy: 1.0000
Epoch 81/100
74/74 [==============================] - 1s 11ms/step - loss: 7.5229e-04 - accuracy: 0.9996
Epoch 82/100
74/74 [==============================] - 1s 12ms/step - loss: 0.0020 - accuracy: 0.9996ETA: 0s - loss:
Epoch 83/100
74/74 [==============================] - 1s 10ms/step - loss: 0.0015 - accuracy: 0.9996
Epoch 84/100
74/74 [==============================] - 1s 14ms/step - loss: 0.0013 - accuracy: 0.9996ETA: 0s - loss: 0.0020 - 
Epoch 85/100
74/74 [==============================] - 1s 15ms/step - loss: 0.0015 - accuracy: 0.9996ETA: 1s - los
Epoch 86/100
74/74 [==============================] - 1s 12ms/step - loss: 5.3533e-04 - accuracy: 1.0000
Epoch 87/100
74/74